In [1]:
meta_prompt = """
You are an expert in prompt optimization for fact verification and multi-hop reasoning tasks. 
Your goal is to improve the end-to-end performance of a claim verification system that operates on the **HoVer benchmark** — 
a dataset designed to test multi-hop evidence retrieval and reasoning over Wikipedia.

====================
🏗️ SYSTEM DESCRIPTION
====================
The HoVer system verifies factual claims by chaining together several reasoning and retrieval components.

1. **Input Claim:** 
   A natural-language factual claim (question) that must be verified as either SUPPORTED, NOT_SUPPORTED, or NOT ENOUGH INFO.

2. **Multi-Hop Query Generation and Retrieval:**
   - The system performs three iterative hops of reasoning and retrieval:
     - Hop 1 → query_1 → passages_1 → summary_1
     - Hop 2 → query_2 → passages_2 → summary_2
     - Hop 3 → query_3 → passages_3 → summary_3
   - Each query is executed by a **static BM25 retriever** that searches the Wikipedia abstracts corpus:

stemmer = Stemmer.Stemmer("english")
retriever = bm25s.BM25.load("/Users/priyanjindal/prompt-learning/benchmarks/hover/wiki17_abstracts",
corpus_name="wiki17_abstracts_corpus.jsonl",
load_corpus=True)
corpus = retriever.corpus

def search(query: str, k: int) -> list[dict]:
    tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
    results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    formatted_results = []
    for doc in results[0]:
        text = doc['text']
        if " | " not in text:
            return []
        title, content = text.split(" | ", 1)
        formatted_results.append({"title": title, "content": content})
    return formatted_results

   - The retriever is **static**; it cannot be trained or modified.
   - Improvements must come from **better query, summary, and reasoning prompts** that guide the model to retrieve the right evidence.

3. **Summarization (Evidence Aggregation):**
   - Each hop produces summaries ({summary_1}, {summary_2}, {summary_3}) that consolidate evidence from retrieved passages.
   - These summaries feed into the next query, so factual precision and context retention are critical.

4. **Final Verdict Generation:**
   - After three hops, the model produces {final_answer}, classifying the claim as:
     - **SUPPORTED**
     - **NOT_SUPPORTED**
     - **NOT ENOUGH INFO**

====================
🎯 OPTIMIZATION OBJECTIVE
====================
You are optimizing **all prompts together** — the entire reasoning chain — including:
- Query generation prompts
- Summarization prompts
- Final verdict prompt

Your task is to propose improved versions that work *coherently* across hops.  
The goal is to maximize factual accuracy, evidence recall, and logical consistency across the whole pipeline.

Specifically, your improved prompt set should:
- Encourage **entity-complete and relation-aware queries** that retrieve all supporting evidence.
- Ensure **summaries** preserve factual links between entities and accurately reflect retrieved passages.
- Guide the **final verdict** toward the correct SUPPORTED/NOT_SUPPORTED/NOT ENOUGH INFO classification based on evidence strength.
- Maintain tight alignment between hops — information extracted or clarified in earlier hops should directly inform later hops.

The retriever is fixed — optimization must come from *how the model expresses reasoning and structures retrieval requests.*

====================
📄 YOUR INPUTS
====================
Below are the **current baseline prompts** for each module in the system, followed by example runs and evaluation feedback.

************* start prompts *************
{baseline_prompt}
************* end prompts *************

************* start example data *************
{examples}
************* end example data *************

HERE ARE SOME ANNOTATIONS THAT MAY BE HELPFUL:
{annotations}

====================
🔧 FINAL INSTRUCTIONS
====================
Iterate on the entire prompt set and produce **new improved versions** that:
- Preserve all variable placeholders (e.g., {question}, {query_1}, {summary_2}, etc.).
- Strengthen inter-module coordination — ensure each hop builds meaningfully on prior ones.
- Promote explicit reasoning about factual relationships, entities, and temporal or causal links.
- Encourage concise but evidence-rich summaries that enhance retrieval quality in later hops.
- Maintain consistent formatting and output schema across all modules.
- Optionally include a few short, high-quality examples to illustrate improved behavior.
- Return the new prompt set in the **same structure and formatting** as the baseline prompt block.

Do **not** wrap anything other than variable names in curly braces.  
Do **not** modify or remove any output-format sections from the original prompts.

NEW PROMPTS:
"""


In [2]:
from application import run_pipeline
import pandas as pd
from hover_evaluate import compute_attach_correctness, attach_evals


def run_train(prompts, df):
    df = run_pipeline(prompts, df)
    df, accuracy = compute_attach_correctness(df)
    df = attach_evals(df)
    return df, accuracy

def run_dev(prompts, df):
    df = run_pipeline(prompts, df)
    df, accuracy = compute_attach_correctness(df)
    return df, accuracy



/opt/anaconda3/envs/hover-benchmark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pathlib import Path
import sys
project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer
import ast

def parse_prompts(s: str) -> dict:
    # First parse: removes the outer quotes if present
    obj = ast.literal_eval(s)
    # If the first parse yielded a string (i.e., still quoted), parse again
    if isinstance(obj, str):
        obj = ast.literal_eval(obj)
    if not isinstance(obj, dict):
        raise ValueError("Parsed object is not a dict")
    return obj

def optimize_prompts(prompts, train_df, meta_prompt):
    prompts_concatenated = str(prompts)
    optimizer = PromptLearningOptimizer(
        prompt=prompts_concatenated,
        model_choice="gpt-5",
        meta_prompt = meta_prompt
    )
    optimized_prompts = optimizer.optimize(
        dataset=train_df,
        output_column="final_answer",
        feedback_columns=["ground_truth_wikipedia_titles", "ground_truth_label", "evaluation"],
    )
    return parse_prompts(optimized_prompts)


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
prompts = {
    "create_query_1_prompt": "Given the fields {claim}, produce the field 'query_1'.",
    "summarize_1_prompt": "Given the fields {claim}, {passages_1}, produce the field 'summary_1'.",
    "create_query_2_prompt": "Given the fields {claim}, {summary_1}, produce the field 'query_2'.",
    "summarize_2_prompt": "Given the fields {claim}, {summary_1}, {passages_2}, produce the field 'summary_2'.",
    "create_query_3_prompt": "Given the fields {claim}, {summary_1}, {summary_2}, produce the field 'query_3'.",
    "summarize_3_prompt": "Given the fields {claim}, {summary_1}, {summary_2}, {passages_3}, produce the field 'summary_3'.",
    "final_answer_prompt": "Given the fields {claim}, {summary_1}, {summary_2}, {summary_3}, return either 'SUPPORTED' or 'NOT_SUPPORTED'."
}

train_df = pd.read_json("hover_train_release_v1.1.json")
train_df = train_df.sample(150, random_state=42)

dev_df = pd.read_json("hover_dev_release_v1.1.json")
dev_df = dev_df.sample(300, random_state=42)





In [7]:
results = []

for i in range(5):
    train_df_run, train_accuracy = run_train(prompts, train_df)
    print(train_accuracy)

    optimized_prompts = optimize_prompts(prompts, train_df_run, meta_prompt)
    
    dev_df_run, dev_accuracy = run_dev(optimized_prompts, dev_df)
    print(dev_accuracy)

    results.append({
        "iteration": i,
        "train_accuracy": train_accuracy,
        "dev_accuracy": dev_accuracy,
        "prompts": prompts,                # before optimization
        "optimized_prompts": optimized_prompts,  # after optimization
    })
    
    prompts = optimized_prompts



llm_generate |█████████▉| 149/150 (99.3%) | ⏳ 00:36<00:00 | 14.61it/s 

Exception in worker on attempt 1: raised APITimeoutError('Request timed out.')
Requeuing...


llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:20<00:00 |  7.40it/s
llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:19<00:00 |  7.52it/s
llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:18<00:00 |  8.11it/s
llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:15<00:00 |  9.46it/s
llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:18<00:00 |  8.09it/s
llm_generate |██████████| 150/150 (100.0%) | ⏳ 00:15<00:00 |  9.69it/s
                                                                       
                                                                       
llm_generate |██████████| 150/150 (100.0%) | ⏳ 03:31<00:00 |  2.64s/it

Exception in worker on attempt 1: raised APITimeoutError('Request timed out.')
Requeuing...


0.6666666666666666
['claim', 'uid', 'ground_truth_label', 'ground_truth_wikipedia_titles', 'query_1', 'passages_1', 'summary_1', 'query_2', 'passages_2', 'summary_2', 'query_3', 'passages_3', 'summary_3', 'final_answer', 'correctness', 'evaluation']

🔧 Creating batches with 128,000 token limit
📊 Processing 150 examples in 4 batches
   ✅ Batch 1/4: Optimized
   ✅ Batch 2/4: Optimized
   ✅ Batch 3/4: Optimized
   ✅ Batch 4/4: Optimized


...

Exception in worker on attempt 1: raised APITimeoutError('Request timed out.')
Requeuing...







































































































































































































































































llm_generate |██████████| 300/300 (100.0%) | ⏳ 00:29<00:00 | 10.20it/s





































































































































































































































































































































































































                                                             

                                                                 
                                                                       


llm_generate |██████████| 150/150 (100.0%) | ⏳ 22:17<00:00 |  4.43s/it


   

Exception in worker on attempt 1: raised APITimeoutError('Request timed out.')
Requeuing...





























































llm_generate |██████████| 150/150 (100.0%) | ⏳ 22:32<00:00 |  9.02s/it
llm_generate |██████████| 150/150 (100.0%) | ⏳ 20:30<00:00 |  8.20s/it
llm_generate |██████████| 300/300 (100.0%) | ⏳ 03:49<00:00 |  1.31it/s
llm_generate |██████████| 300/300 (100.0%) | ⏳ 01:34<00:00 |  3.17it/s










































































































































































































































































































































llm_generate |██████████| 300/300 (100.0%) | ⏳ 00:29<00:00 | 10.17it/s
llm_generate |██████████| 300/300 (100.0%) | ⏳ 01:32<00:00 |  3.25it/s
llm_generate |██████████| 300/300 (100.0%) | ⏳ 00:28<00:00 | 10.58it/s

0.5633333333333334


In [8]:
results

[{'iteration': 0,
  'train_accuracy': np.float64(0.6666666666666666),
  'dev_accuracy': np.float64(0.47),
  'prompts': {'create_query_1_prompt': "Given the fields {claim}, produce the field 'query_1'.",
   'summarize_1_prompt': "Given the fields {claim}, {passages_1}, produce the field 'summary_1'.",
   'create_query_2_prompt': "Given the fields {claim}, {summary_1}, produce the field 'query_2'.",
   'summarize_2_prompt': "Given the fields {claim}, {summary_1}, {passages_2}, produce the field 'summary_2'.",
   'create_query_3_prompt': "Given the fields {claim}, {summary_1}, {summary_2}, produce the field 'query_3'.",
   'summarize_3_prompt': "Given the fields {claim}, {summary_1}, {summary_2}, {passages_3}, produce the field 'summary_3'.",
   'final_answer_prompt': "Given the fields {claim}, {summary_1}, {summary_2}, {summary_3}, return either 'SUPPORTED' or 'NOT_SUPPORTED'."},
  'optimized_prompts': {'create_query_1_prompt': 'Task: Using {claim}, produce the field \'query_1\' as a sin